# 📥 Notebook 1: Coleta de Dados - Reddit

Este notebook demonstra o processo de coleta de dados de redes sociais (Reddit) para análise de sentimento em tópicos específicos.

## 🎯 Objetivos
- Configurar e testar coletores de dados
- Coletar posts sobre um tópico específico
- Validar qualidade dos dados coletados
- Preparar dados para pré-processamento

## 📋 Pré-requisitos
- Credenciais do Reddit configuradas em `.env`
- Ambiente Python com dependências instaladas
- Configuração do tópico em `config/topic.yaml`

In [5]:
# Imports e configuração inicial
import sys
import os
import pandas as pd
import yaml
from datetime import datetime
from pathlib import Path
import logging

# Adicionar src ao path
sys.path.append('../src')

# Configurar logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Verificar estrutura do projeto
project_root = Path("..").resolve()
print(f"📁 Diretório do projeto: {project_root}")
print(f"📂 Estrutura principal:")
for item in ["config", "src", "data", "scripts"]:
    path = project_root / item
    status = "✅" if path.exists() else "❌"
    print(f"  {status} {item}/")

print("\n🔧 Python path:")
for p in sys.path[-3:]:
    print(f"  - {p}")

📁 Diretório do projeto: /home/diego/Documentos/disciplina-mestrado
📂 Estrutura principal:
  ✅ config/
  ✅ src/
  ✅ data/
  ✅ scripts/

🔧 Python path:
  - /home/diego/Documentos/disciplina-mestrado/venv/lib/python3.11/site-packages
  - ../src
  - ../src


In [6]:
# Carregar configuração do tópico
config_path = project_root / "config" / "topic.yaml"

try:
    with open(config_path, 'r', encoding='utf-8') as f:
        config = yaml.safe_load(f)
    
    print("📋 Configuração carregada:")
    print(f"  🎯 Tópico: {config['topic']}")
    print(f"  🔍 Keywords: {config['keywords']}")
    print(f"  📊 Limites:")
    print(f"    - Reddit: {config['limits']['reddit']}")
    print(f"  🔧 Filtros:")
    print(f"    - Comprimento: {config['filters']['min_length']}-{config['filters']['max_length']}")
    print(f"    - Idioma: {config['filters']['language']}")
    
except FileNotFoundError:
    print(f"❌ Arquivo de configuração não encontrado: {config_path}")
    print("💡 Execute: cp config/topic.yaml.example config/topic.yaml")
except Exception as e:
    print(f"❌ Erro ao carregar configuração: {e}")

📋 Configuração carregada:
  🎯 Tópico: Taxas Trump Brasil
  🔍 Keywords: ['trump', 'tarifa', 'taxa', 'imposto', 'brasil', 'eua', 'estados unidos', 'donald trump', 'tarifas', 'protecionismo', 'guerra comercial', 'economia', 'comércio exterior', 'exportação', 'importação', 'brics', 'dólar', 'moeda', 'sanções', 'retaliação']
  📊 Limites:
    - Reddit: 8000
  🔧 Filtros:
    - Comprimento: 15-500
    - Idioma: pt


In [7]:
# Testar importação dos scrapers
print("\n🔍 Testando importação dos scrapers:")
try:
    from scrapers.reddit import RedditScraper  
    print("✅ RedditScraper importado com sucesso")
except ImportError as e:
    print(f"❌ Erro ao importar RedditScraper: {e}")

try:
    from utils.config import load_config, setup_logging
    print("✅ Utilitários importados com sucesso")
except ImportError as e:
    print(f"❌ Erro ao importar utilitários: {e}")

# Verificar se todos os arquivos necessários existem
required_files = [
    "src/scrapers/reddit.py", 
    "src/utils/config.py",
    ".env.example"
]

print("\n📄 Verificando arquivos necessários:")
for file_path in required_files:
    full_path = project_root / file_path
    status = "✅" if full_path.exists() else "❌"
    print(f"  {status} {file_path}")


🔍 Testando importação dos scrapers:
✅ RedditScraper importado com sucesso
✅ Utilitários importados com sucesso

📄 Verificando arquivos necessários:
  ✅ src/scrapers/reddit.py
  ✅ src/utils/config.py
  ✅ .env.example


## 🔴 Teste de Coleta: Reddit

Vamos testar a coleta de dados do Reddit usando PRAW. **Importante**: Este teste requer credenciais configuradas no arquivo `.env`.

In [8]:
# Verificar credenciais do Reddit
import os
from dotenv import load_dotenv

# Carregar variáveis do .env
env_path = project_root / ".env"
if env_path.exists():
    load_dotenv(env_path)
    print("✅ Arquivo .env encontrado")
else:
    print("⚠️  Arquivo .env não encontrado")
    print("💡 Execute: cp .env.example .env e configure as credenciais")

# Verificar se as credenciais estão configuradas
required_vars = ["REDDIT_ID", "REDDIT_SECRET", "REDDIT_AGENT", "REDDIT_USERNAME", "REDDIT_PASSWORD"]
missing_vars = []

print("\n🔐 Verificando credenciais do Reddit:")
for var in required_vars:
    value = os.getenv(var)
    if value:
        # Mostrar apenas primeiros caracteres para segurança
        display_value = value[:4] + "..." if len(value) > 4 else value
        print(f"  ✅ {var}: {display_value}")
    else:
        print(f"  ❌ {var}: não configurado")
        missing_vars.append(var)

if missing_vars:
    print(f"\n⚠️  Variáveis faltando: {missing_vars}")
    print("💡 Configure no arquivo .env antes de continuar")
else:
    print("\n✅ Todas as credenciais estão configuradas!")

✅ Arquivo .env encontrado

🔐 Verificando credenciais do Reddit:
  ✅ REDDIT_ID: HN2e...
  ✅ REDDIT_SECRET: Has5...
  ✅ REDDIT_AGENT: sent...
  ✅ REDDIT_USERNAME: bian...
  ✅ REDDIT_PASSWORD: D13g...

✅ Todas as credenciais estão configuradas!


In [10]:
# Teste de coleta Reddit com limite pequeno
try:
    from scrapers.reddit import RedditScraper
    
    # Verificar se as credenciais estão disponíveis
    if not missing_vars:
        print("🔍 Testando coleta do Reddit...")
        
        # Configurações de teste
        test_subreddits = config['reddit']['subreddits'][:1]  # Usar apenas 1 subreddit para teste
        test_limit = 5  # Limite pequeno para teste
        
        print(f"📊 Subreddits: {test_subreddits}")
        print(f"📊 Limite: {test_limit} posts por subreddit")
        
        # Inicializar scraper com a configuração
        reddit_scraper = RedditScraper(config)
        
        # Realizar coleta de teste usando o método correto
        reddit_data = reddit_scraper.scrape(
            keywords=config['keywords'][:2],
            limit=test_limit
        )
        
        if reddit_data:
            print(f"✅ Coleta bem-sucedida! {len(reddit_data)} posts coletados")
            
            # Converter para DataFrame para análise
            df_reddit = pd.DataFrame(reddit_data)
            print(f"📊 Colunas: {list(df_reddit.columns)}")
            print(f"📝 Exemplo de post:")
            if len(df_reddit) > 0:
                print(f"  Título: {df_reddit.iloc[0]['title'][:100]}...")
                print(f"  Data: {df_reddit.iloc[0]['timestamp']}")
                print(f"  Subreddit: {df_reddit.iloc[0].get('subreddit', 'N/A')}")
                print(f"  Score: {df_reddit.iloc[0].get('score', 'N/A')}")
        else:
            print("⚠️  Nenhum post coletado. Verifique as keywords ou subreddits.")
            
    else:
        print("❌ Não é possível testar Reddit sem credenciais configuradas")
        print("💡 Configure o arquivo .env primeiro")
        
except Exception as e:
    print(f"❌ Erro na coleta do Reddit: {e}")
    print("💡 Verifique se o praw está instalado: pip install praw")

2025-07-10 20:00:17,943 - INFO - Scraping Reddit nos subreddits: ['brasil', 'brasilivre', 'politica', 'brasildob', 'circojeca', 'desabafos', 'investimentos', 'farialimabets', 'coronabr', 'conversas', 'economiadobrasil', 'politics', 'worldnews', 'news', 'economics', 'geopolitics', 'conservative', 'liberal', 'askreddit', 'unpopularopinion', 'changemyview', 'outoftheloop', 'explainlikeimfive']
2025-07-10 20:00:17,943 - INFO - Keywords: ['trump', 'tarifa']
2025-07-10 20:00:17,944 - INFO - Limite total: 5 posts
2025-07-10 20:00:17,944 - INFO - Buscando em r/brasil (quota: 0)...
2025-07-10 20:00:17,945 - INFO -   Buscando posts 'hot' em r/brasil


🔍 Testando coleta do Reddit...
📊 Subreddits: ['brasil']
📊 Limite: 5 posts por subreddit


2025-07-10 20:00:19,210 - INFO - Coletados 2 posts de r/brasil
2025-07-10 20:00:19,919 - INFO - Buscando em r/brasilivre (quota: 0)...
2025-07-10 20:00:19,920 - INFO -   Buscando posts 'hot' em r/brasilivre
2025-07-10 20:00:20,929 - INFO - Coletados 1 posts de r/brasilivre
2025-07-10 20:00:21,539 - INFO - Buscando em r/politica (quota: 0)...
2025-07-10 20:00:21,539 - INFO -   Buscando posts 'hot' em r/politica
2025-07-10 20:00:22,530 - INFO - Coletados 1 posts de r/politica
2025-07-10 20:00:23,274 - INFO - Buscando em r/brasildob (quota: 10)...
2025-07-10 20:00:23,275 - INFO -   Buscando posts 'hot' em r/brasildob
2025-07-10 20:00:24,168 - INFO -   Buscando posts 'top' em r/brasildob
2025-07-10 20:00:25,012 - INFO -   Buscando posts 'new' em r/brasildob
2025-07-10 20:00:26,004 - INFO -   Buscando posts 'rising' em r/brasildob
2025-07-10 20:00:27,081 - INFO -   Buscando posts populares gerais em r/brasildob
2025-07-10 20:00:27,431 - INFO - Coletados 6 posts de r/brasildob
2025-07-10 20:

✅ Coleta bem-sucedida! 5 posts coletados
📊 Colunas: ['platform', 'id', 'text', 'title', 'selftext', 'timestamp', 'user_hash', 'username', 'score', 'upvote_ratio', 'num_comments', 'subreddit', 'url', 'search_method', 'search_keyword', 'collected_at']
📝 Exemplo de post:
  Título: Chantagem rasteira de Trump não passará...
  Data: 2025-07-10 12:44:45
  Subreddit: brasil
  Score: 70


## 🚀 Coleta Completa

Uma vez que os testes funcionaram, podemos executar a coleta completa usando os limites definidos na configuração.

In [ ]:
# Executar coleta completa usando o script collector.py
import subprocess
import json
from datetime import datetime

# Definir parâmetros da coleta - com tratamento para Twitter não implementado
twitter_limit = config.get('limits', {}).get('twitter', 0)  # Default 0 se não existir
reddit_limit = config['limits']['reddit']
topic = config['topic']

print(f"🎯 Iniciando coleta completa para: {topic}")
print(f"📊 Limites: Twitter={twitter_limit} (não implementado), Reddit={reddit_limit}")

# Avisar sobre limitação do Twitter
if twitter_limit > 0:
    print("⚠️  Aviso: Coleta do Twitter não está implementada nesta versão")
    print("📊 Apenas dados do Reddit serão coletados")
else:
    print("✅ Configurado para coleta apenas do Reddit")

# Criar timestamp para esta execução
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = project_root / "data" / "raw" / timestamp

print(f"📁 Salvando em: {output_dir}")

# Comando para executar o collector
cmd = [
    "python", 
    str(project_root / "scripts" / "collector.py"),
    "--config", str(project_root / "config" / "topic.yaml"),
    "--limit-twitter", str(twitter_limit),
    "--limit-reddit", str(reddit_limit),
    "--output-dir", str(output_dir),
    "--verbose"
]

print(f"🔧 Comando: {' '.join(cmd)}")
print("\n" + "="*50)
print("🚀 EXECUTANDO COLETA...")
print("="*50)

try:
    # Executar o comando
    result = subprocess.run(cmd, capture_output=True, text=True, cwd=str(project_root))
    
    print("📤 SAÍDA:")
    print(result.stdout)
    
    if result.stderr:
        print("⚠️  ERROS/AVISOS:")
        print(result.stderr)
    
    if result.returncode == 0:
        print("✅ Coleta concluída com sucesso!")
        
        # Verificar arquivos gerados
        if output_dir.exists():
            files = list(output_dir.glob("*.csv"))
            print(f"📄 Arquivos gerados: {len(files)}")
            for file in files:
                size_mb = file.stat().st_size / (1024*1024)
                print(f"  - {file.name}: {size_mb:.2f} MB")
                
                # Mostrar preview dos dados se for arquivo do Reddit
                if file.name.startswith('reddit_'):
                    try:
                        import pandas as pd
                        df = pd.read_csv(file)
                        print(f"    📊 {len(df)} posts do Reddit coletados")
                        if len(df) > 0:
                            print(f"    📅 Período: {df['timestamp'].min()} até {df['timestamp'].max()}")
                            subreddit_dist = df['subreddit'].value_counts().head(5).to_dict()
                            print(f"    🔗 Top subreddits: {subreddit_dist}")
                    except Exception as e:
                        print(f"    ⚠️  Erro ao ler preview: {e}")
        else:
            print("⚠️  Diretório de saída não encontrado")
    else:
        print(f"❌ Coleta falhou com código: {result.returncode}")
        
except Exception as e:
    print(f"❌ Erro ao executar coleta: {e}")

print("\n" + "="*50)

🎯 Iniciando coleta completa para: Taxas Trump Brasil
📊 Limites: Twitter=0 (não implementado), Reddit=8000
✅ Configurado para coleta apenas do Reddit
📁 Salvando em: /home/diego/Documentos/disciplina-mestrado/data/raw/20250710_200409
🔧 Comando: python /home/diego/Documentos/disciplina-mestrado/scripts/collector.py --config /home/diego/Documentos/disciplina-mestrado/config/topic.yaml --limit-twitter 0 --limit-reddit 8000 --output-dir /home/diego/Documentos/disciplina-mestrado/data/raw/20250710_200409 --verbose

🚀 EXECUTANDO COLETA...
